In [ ]:
# Install and import relevant libraries
!python -m easy_install ../input/compiledlucene/bk/lucene-8.1.1-py3.6-linux-x86_64.egg
!cp -r ../input/compiledlucene/bk/JCC-3.7-py3.6-linux-x86_64.egg /opt/conda/lib/python3.6/site-packages/
import sys
sys.path
sys.path.append('/opt/conda/lib/python3.6/site-packages/JCC-3.7-py3.6-linux-x86_64.egg')
sys.path.append('/opt/conda/lib/python3.6/site-packages/lucene-8.1.1-py3.6-linux-x86_64.egg')

In this kernel, we provide an installation/ configuration/ compilation package for PyLucene 8.1.1 as an external data called “compiledlucene” which provide all required software dependencies for installation and deployment of PyLucene. 

In [ ]:
import sys, os, lucene, threading, time
from datetime import datetime
from java.nio.file import Paths
from org.apache.lucene.analysis.miscellaneous import LimitTokenCountAnalyzer
from org.apache.lucene.document import Document, Field, FieldType
from org.apache.lucene.index import FieldInfo, IndexWriter, IndexWriterConfig, IndexOptions
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.index import DirectoryReader
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.store import SimpleFSDirectory
from org.apache.lucene.search import IndexSearcher
import pandas as pd

LUCENE_INDEX_DIR = "documentLevel"
LUCENE_BASE_DIR = "/kaggle/working"
#COVID_FULLTEXT_DF = "../input/covidfulltext/metadata_and_fulltext_2020-04-17.csv"

In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import bs4 as bs
import urllib.request
from time import sleep
import os
from IPython.display import display, HTML

# **Map abstract and respective JSON data into one data frame**

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from IPython.core.display import display, HTML
import pandas as pd
import torch
from transformers import *
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
# keep only documents with covid -cov-2 and cov2
def search_focus(df):
    dfa = df[df['abstract'].str.contains('covid')]
    dfb = df[df['abstract'].str.contains('-cov-2')]
    dfc = df[df['abstract'].str.contains('cov2')]
    dfd = df[df['abstract'].str.contains('ncov')]
    frames=[dfa,dfb,dfc,dfd]
    df = pd.concat(frames)
    df=df.drop_duplicates(subset='title', keep="first")
    return df

# load the meta data from the CSV file
#usecols=['title','journal','abstract','authors','doi','publish_time','sha','full_text_file']
df=pd.read_csv('/kaggle/input/CORD-19-research-challenge/metadata.csv')
print ('ALL CORD19 articles',df.shape)
#fill na fields
df=df.fillna('no data provided')
#drop duplicate titles
df = df.drop_duplicates(subset='title', keep="first")
#keep only 2020 dated papers
df=df[df['publish_time'].str.contains('2020')]
# convert abstracts to lowercase
df["abstract"] = df["abstract"].str.lower()+df["title"].str.lower()
#show 5 lines of the new dataframe
#df=search_focus(df)
print ('Keep only COVID-19 related articles',df.shape)

import os
import json
from pprint import pprint
from copy import deepcopy
import math


def format_body(body_text):
    texts = [(di['section'], di['text']) for di in body_text]
    texts_di = {di['section']: "" for di in body_text}
    
    for section, text in texts:
        texts_di[section] += text

    body = ""

    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"
    
    return body


for index, row in df.iterrows():
    #print (row['pdf_json_files'])
    if 'no data provided' not in row['pdf_json_files'] and os.path.exists('/kaggle/input/CORD-19-research-challenge/'+row['pdf_json_files'])==True:
        with open('/kaggle/input/CORD-19-research-challenge/'+row['pdf_json_files']) as json_file:
            #print ('in loop')
            data = json.load(json_file)
            body=format_body(data['body_text'])
            #print (body)
            body=body.replace("\n", " ")
            text=row['abstract']+' '+body.lower()
            df.loc[index, 'abstract'] =text

df=df.drop(['pdf_json_files'], axis=1)
df=df.drop(['sha'], axis=1)
df.head()

In [ ]:
df.shape

In [ ]:
#basic seach based result start

In [ ]:
df1= df[['title','publish_time','journal','url','abstract','doi','cord_uid']]
#Make a copy to work with
df_relevant=df1.copy()

In [ ]:
#separate each word in the column: abstract for browsing
df_relevant['words'] = df_relevant.abstract.str.strip().str.split('[\W_]+')

In [ ]:
#separate words in the abstract column and create a new column
Abstracts1 = df_relevant[df_relevant.words.str.len() > 0]
Abstracts1.head(2)

In [ ]:
Abstracts1.shape

In [ ]:
# saving the Abstracts Table (dataframe) 
Abstracts1.to_csv('All_Abstracts.csv') 

In [ ]:
assay = Abstracts1[Abstracts1['abstract'].str.contains('assay')]
assay.to_csv('Assay.csv')

In [ ]:
methods=['RT-iiPCR', 'Isothermal amplification', 'Antigen EIA', 'SENSR (Novel pathogen diagnostic technique)', 'rRT-PCR', 'Roche Platform',
 'ELISA', 'rapid test IgG/IgM', 'Microarray', 'One Step rRT-PCR', 'specific-HCoV', 'RT-LAMP,', 'Rapid antigen test', 'monoplex', 'Antigen Detection test',
 'Euroimmun IgA ELISA', 'HTS', 'Ultrasound', 'Serology', 'Penn-RAMP', 'Cell culture', 'pan-HCoV', 'RT-LAMP', 'LAMP', 'Rapid antigen test ', 'Multiple discussed',
 'COVID-19 IgM/IgG Rapid Test of BioMedomics', 'Antigen EIA; ', 'dPCR', 'Lateral Flow Assay', 'RT-PCR (Novel Procedure)', 'multiplex', 'Roche cobas SARS-CoV-2 assay',
 'Cepheid Xpert Xpress SARS-CoV-2 assay', 'mini-PCR', 'Euroimmun IgG ELISA', 'Antigen IFA', 'Serology (IgM & IgG Ab Eval)', 'Spiral CT', 'CXR',
 'Wantai SARS-CoV-2 Total Antibody ELISA', 'VivaDiag COVID-19 IgM / IgG Rapid Test lateral flow immunoassay (LFIA)', 'Lateral Flow Antigen Detection',
 'One step RT-PCR', 'All-in-One Dual CRISPR-Cas12a (AIOD-CRISPR)', 'mNGS', 'CRISPR', 'NAAT (PCR)', 'CT', 'NAAT', 'rapid serological test Viva-Diag analyzingCOVID-19 associated-IgG/IgM.',
 'Serology ', 'RT-dPCR', 'Pixelated colorimetric nucleic acid assay', 'COVID-19 FET sensor', 'RT-PCR ', 'RT-PCR', 'Microfluidic Devices',
 'RT-PCR, Serology', 'rRT-PCR kit (QIAStat-Dx Respiratory Panel)', 'iLAMP']

In [ ]:
detection_Method = Abstracts1[Abstracts1['abstract'].str.contains('|'.join(methods),case=False)]
detection_Method.shape
detection_Method.to_csv('detection_Method.csv')

In [ ]:
FDA=Abstracts1[Abstracts1['abstract'].str.contains('FDA',case=False)]
FDA.shape
FDA.to_csv('FDA.csv')

In [ ]:
studyType=['Simulation', 'Simulation Study', 'Simlation Study', 'Clinical trial', 'Retrospective Cohort', 'Expert Review', 'Review', 'Proof of concept', 'Investigative Study', 'Simulation study', 'Prospective Cohort', 'Clinical Study', 'Clinical Trial', 'Case Report', 'Systematic Review',
 'Retrospective Cohort Study']

In [ ]:
study_type=Abstracts1[Abstracts1['abstract'].str.contains('|'.join(studyType),case=False)]
study_type.shape
study_type.to_csv('study_type.csv')

In [ ]:
Measure_Accuracy=['sensitivity','specifity','positive predictive value','negative predicitve value','accuracy',]
Measure_of_Testing_Accuracy=Abstracts1[Abstracts1['abstract'].str.contains('|'.join(Measure_Accuracy),case=False)]
print(Measure_of_Testing_Accuracy.shape)
Measure_of_Testing_Accuracy.to_csv('Measure_of_Testing_Accuracy.csv')

In [ ]:
from IPython.display import FileLink
FileLink(r'Measure_of_Testing_Accuracy.csv')

# Final csv merged based on cord_uid

In [ ]:
df_all = pd.read_csv('All_Abstracts.csv')
df_assay = pd.read_csv('Assay.csv')
df_fda = pd.read_csv('FDA.csv')
df_accuracy =pd.read_csv('Measure_of_Testing_Accuracy.csv')
df_detection = pd.read_csv('detection_Method.csv')
df_study_type=pd.read_csv('study_type.csv')

In [ ]:
dfs = [df_all,df_assay,df_fda,df_accuracy,df_detection,df_study_type]

In [ ]:
df_final_1 = pd.merge(df_all, df_assay, on='cord_uid', how='left')
df_final_1=df_final_1.drop(['Unnamed: 0_y', 'title_y','publish_time_y', 'journal_y', 'url_y', 'abstract_y', 'doi_y',
       'words_y'], axis=1)
df_final_2 = pd.merge(df_final_1, df_fda, on='cord_uid', how='left')
df_final_2=df_final_2.drop(['Unnamed: 0', 'title','publish_time', 'journal', 'url', 'abstract', 'doi','words'], axis=1)
df_final_3 = pd.merge(df_final_2, df_accuracy, on='cord_uid', how='left')
df_final_3=df_final_3.drop(['Unnamed: 0', 'title','publish_time', 'journal', 'url', 'abstract', 'doi','words'], axis=1)
df_final_4 = pd.merge(df_final_3, df_detection, on='cord_uid', how='left')
df_final_4=df_final_4.drop(['Unnamed: 0', 'title','publish_time', 'journal', 'url', 'abstract', 'doi','words'], axis=1)
df_final_5 = pd.merge(df_final_4, df_study_type, on='cord_uid', how='left')
df_final_New=df_final_5.drop(['Unnamed: 0', 'title','publish_time', 'journal', 'url', 'abstract', 'doi','words'], axis=1)

#df_left

In [ ]:
df_final_New.columns

In [ ]:
df_final_New.shape

In [ ]:
df_final_New.columns = ['Unnamed: 0', 'title', 'publish_time', 'journal', 'url','abstract', 'doi', 'cord_uid', 'words']

In [ ]:
df_final_New.columns

In [ ]:
df_final_New.to_csv('df_final.csv')

In [ ]:
from IPython.display import FileLink
FileLink(r'df_final.csv')

In [ ]:
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='cord_uid'), dfs)

In [ ]:
df_final.shape

In [ ]:
df_final.to_csv('df_final.csv')

In [ ]:
from IPython.display import FileLink
FileLink(r'df_final.csv')

In [ ]:
import functools

def remove_stopwords(text,stopwords):
    text = "".join(c for c in text if c not in ('!','.',',','?','(',')','-'))
    text_tokens = word_tokenize(text)
    #remove stopwords
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    str1=''
    str1=' '.join(word for word in tokens_without_sw)
    return str1

### spacy score sentence
def score_sentence(search,sentence):
        main_doc=nlp(sentence)
        search_doc=nlp(search)
        sent_score=main_doc.similarity(search_doc)
        return sent_score

In [ ]:
# custom sentence score
def score_sentence_prob(search,sentence,focus):
    keywords=search.split()
    sent_parts=sentence.split()
    word_match=0
    missing=0
    for word in keywords:
        word_count=sent_parts.count(word)
        word_match=word_match+word_count
        if word_count==0:
            missing=missing+1
    percent = 1-(missing/len(keywords))
    final_score=abs((word_match/len(sent_parts)) * percent)
    if missing==0:
        final_score=final_score+1
    if focus not in sentence:
        final_score=0
    return final_score

# BERT pretrained question answering module
def answer_question(question,text, model,tokenizer):
    input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
    input_ids = tokenizer.encode(input_text)
    token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
    start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    #print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))
    answer=(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))
    # show qeustion and text
    #tokenizer.decode(input_ids)
    answer=answer.replace(" ##", "")
    answer=answer.replace(" · ", "·")
    answer=answer.replace(" . ", ".")
    answer=answer.replace(" , ", ",")
    if '[SEP]'in answer or '[CLS]' in answer or answer=='':
        answer='unk'
        
    return answer


In [ ]:
#test on limite dataset
def process_question(df,search,focus):
    print('--------0----------')
    df_table = pd.DataFrame(columns = ['date','study','link','Journal','studytype','sample','method','ObtainedSample','MeasureofTestingAccuracy','Speedofassay','cord_uid','FDAapproval','Addedon','DOI'])
    # focuses to make sure the exact phrase in text
    #df1 = df[df['abstract'].str.contains(focus)]
    # focus to make sure all words in text
    df1=df[functools.reduce(lambda a, b: a&b, (df['abstract'].str.contains(s) for s in search.split()))]
    print('-----1-------',df['abstract'])
    search=remove_stopwords(search,stopwords)
    print('-------2-----',search)
    print('-----df1-------',df1)
    for index, row in df1.iterrows():
        print('-----3------',row)
        sentences = row['abstract'].split('. ')
        pub_sentence=''
        hi_score=0
        study=''
        hi_study_score=0
        for sentence in sentences:
            if len(sentence)>75 and focus in sentence:
                rel_score=score_sentence_prob(search,sentence,focus)
                if rel_score>.02:
                    sentence=sentence.capitalize()
                    if sentence[len(sentence)-1]!='.':
                        sentence=sentence+'.'
                    pub_sentence=pub_sentence+' '+sentence
                    if rel_score>hi_score:
                        hi_score=rel_score
                
        if pub_sentence!='':
            text=row['abstract'][0:100]
            
            question='how many patients or cases were in the study, review or analysis?'
            sample=answer_question(question,text,model,tokenizer)
            sample=sample.replace("#", "")
            sample=sample.replace(" , ", ",")
            if sample=='19' or sample=='' or '[SEP]'in sample:
                sample='unk'
            if len(sample)>50:
                sample='unk'
            sample=sample.replace(" ", "")
            print('-------sample----',sample)

In [ ]:
df_demo = df[:4]

In [ ]:
search ='Diagnosing SARS-COV-2 with antibodies'

In [ ]:
cd = (df['abstract'].str.contains(s) for s in search.split())

In [ ]:
print(*cd)

In [ ]:
df1=df[functools.reduce(lambda a, b: a&b, (df['abstract'].str.contains(s) for s in search.split()))]

In [ ]:
df1

In [ ]:
cd=process_question(df_demo,'Diagnosing SARS-COV-2 with antibodies','diagnostics')

In [ ]:
#dummy test
###################### MAIN PROGRAM ###########################


### focus quesiton with single keyword
#keywords = ['hypertension']
keywords = ['hypertension','diabetes','heart disease','gender','copd','smoking','age','stroke','cerbrovascular','cancer','kidney disease','drinking','tuberculosis','obesity']
#'diabetes','heart disease','male gender','copd','smoking','age','stroke','cerbrovascular','cancer','kidney disease','drinking','tuberculosis','bmi'

q=0

df_all_risk_factors = pd.DataFrame(columns = ["date","study","link","journal","severe","sever sig.","severe age adj.","Severe OR Calculated or Extracted","fatality","fatality sig.","fatality age adj.","Fatality OR Calculated or Extracted","design","sample","risk factor"])

# loop through the list of questions
for keyword in keywords:
    # limit results to severe risk factors
    search_words = keyword+' risk factor severe'
    # get best sentences
    df_table=process_question(df,search_words,keyword)
    df_answers=df_table  
    #print (text)
    #limit the size of the df for the html table
    #df_table=df_table.head(100)
    df_table=df_table.drop_duplicates(subset='study', keep="first")
    df_table = df_table.sort_values(by=['date'], ascending=False)
    
    df_allriskcsv=df_table
    
    #convert df to html
    df_table=HTML(df_table.to_html(escape=False,index=False))
    
    # show the HTML table with responses
    display(df_table)
    
    
    q=q+1
    df_allriskcsv.to_csv(keyword+'_risk_factors.csv',index = False)
    df_all_risk_factors=df_all_risk_factors.append(df_allriskcsv, ignore_index=True)
df_all_risk_factors.to_csv('all_risk_factors.csv',index = False)
print ('done')

In [ ]:
def process_question(df,search,focus):
    df_table = pd.DataFrame(columns = ['date','study','link','Journal','studytype','sample','method','ObtainedSample','MeasureofTestingAccuracy','Speedofassay','cord_uid','FDAapproval','Addedon','DOI'])
    # focuses to make sure the exact phrase in text
    #df1 = df[df['abstract'].str.contains(focus)]
    # focus to make sure all words in text
    df1=df[functools.reduce(lambda a, b: a&b, (df['abstract'].str.contains(s) for s in search))]
    search=remove_stopwords(search,stopwords)
    for index, row in df1.iterrows():
        sentences = row['abstract'].split('. ')
        pub_sentence=''
        hi_score=0
        study=''
        hi_study_score=0
        for sentence in sentences:
            if len(sentence)>75 and focus in sentence:
                rel_score=score_sentence_prob(search,sentence,focus)
                if rel_score>.02:
                    sentence=sentence.capitalize()
                    if sentence[len(sentence)-1]!='.':
                        sentence=sentence+'.'
                    pub_sentence=pub_sentence+' '+sentence
                    if rel_score>hi_score:
                        hi_score=rel_score
                
        if pub_sentence!='':
            text=row['abstract'][0:1000]
            
            question='how many patients or cases were in the study, review or analysis?'
            sample=answer_question(question,text,model,tokenizer)
            sample=sample.replace("#", "")
            sample=sample.replace(" , ", ",")
            if sample=='19' or sample=='' or '[SEP]'in sample:
                sample='unk'
            if len(sample)>50:
                sample='unk'
            sample=sample.replace(" ", "")
            
            question='what type or kind of review study analysis model was used?'
            design=answer_question(question,text,model,tokenizer)
            design=design.replace(" ##", "")
            if '[SEP]'in design or '[CLS]' in design or design=='':
                design='unk'
            
            ### get sever numbers
            question='what is the '+keyword+' HR OR RR AOR hazard odds ratio ()?'
            severe=answer_question(question,text,model,tokenizer)
            df_table.loc[index,'severe']=severe
            
            authors=row["authors"].split(" ")
            link=row['doi']
            title=row["title"]
            score=hi_score
            journal=row["journal"]
            if journal=='':
                journal=row['full_text_file']
            linka='https://doi.org/'+link
            linkb=title
            final_link='<p align="left"><a href="{}">{}</a></p>'.format(linka,linkb)
            #author_link='<p align="left"><a href="{}">{}</a></p>'.format(linka,authors[0]+' et al.')
            #sentence=pub_sentence+' '+author_link
            sentence=pub_sentence
            #sentence='<p fontsize=tiny" align="left">'+sentence+'</p>'
            to_append = [row['publish_time'],title,linka,journal,severe,"-","-","-","-","-","-","-",design,sample,keyword]
            df_length = len(df_table)
            df_table.loc[df_length] = to_append
            
    return df_table

In [ ]:
###################### MAIN PROGRAM ###########################


### focus quesiton with single keyword
#keywords = ['hypertension']
keywords = ['hypertension','diabetes','heart disease','gender','copd','smoking','age','stroke','cerbrovascular','cancer','kidney disease','drinking','tuberculosis','obesity']
#'diabetes','heart disease','male gender','copd','smoking','age','stroke','cerbrovascular','cancer','kidney disease','drinking','tuberculosis','bmi'

q=0

df_all_risk_factors = pd.DataFrame(columns = ["date","study","link","journal","severe","sever sig.","severe age adj.","Severe OR Calculated or Extracted","fatality","fatality sig.","fatality age adj.","Fatality OR Calculated or Extracted","design","sample","risk factor"])

# loop through the list of questions
for keyword in keywords:
    # limit results to severe risk factors
    search_words = keyword+' risk factor severe'
    
    # get best sentences
    df_table=process_question(df,search_words,keyword)
    df_answers=df_table
        
    display(HTML('<h3>'+search_words+'</h3>'))
    
    #print (text)
    
    #limit the size of the df for the html table
    #df_table=df_table.head(100)
    df_table=df_table.drop_duplicates(subset='study', keep="first")
    df_table = df_table.sort_values(by=['date'], ascending=False)
    
    df_allriskcsv=df_table
    
    #convert df to html
    df_table=HTML(df_table.to_html(escape=False,index=False))
    
    # show the HTML table with responses
    display(df_table)
    
    
    q=q+1
    df_allriskcsv.to_csv(keyword+'_risk_factors.csv',index = False)
    df_all_risk_factors=df_all_risk_factors.append(df_allriskcsv, ignore_index=True)
df_all_risk_factors.to_csv('all_risk_factors.csv',index = False)
print ('done')

In [ ]:
# Load indexed metadata 
# final_df = pd.read_csv('../input/covidfulltext/metadata_and_fulltext_2020-04-17.csv')
# final_df.shape

In [ ]:
final_df = pd.read_csv('../input/CORD-19-research-challenge/metadata.csv')
#('../input/covidfulltext/metadata_and_fulltext_2020-04-17.csv')
final_df.shape

In [ ]:
final_df=final_df[:500]

In [ ]:
final_df.shape

In [ ]:
LUCENE_BASE_DIR

In [ ]:
os.path.join(LUCENE_BASE_DIR, LUCENE_INDEX_DIR)

In [ ]:
class IndexFiles(object):
    """Usage: python IndexFiles <doc_directory>"""

    def __init__(self, root, storeDir, analyzer):
        ##print("before store")
        if not os.path.exists(storeDir):
            os.mkdir(storeDir)
        ##print("after store")

        store = SimpleFSDirectory(Paths.get(storeDir))
        ##print(storeDir)
        analyzer = LimitTokenCountAnalyzer(analyzer, 1048576)
        ##print("after analyzer ")

        config = IndexWriterConfig(analyzer)
        ##print("after config")

        config.setOpenMode(IndexWriterConfig.OpenMode.CREATE)
        ##print("before writer")
        writer = IndexWriter(store, config)
        ##print("after writer")
        self.indexDocs(root, writer)
        #indexDocs(root, writer)
        #ticker = Ticker()
        ##print ('commit index')
        #threading.Thread(target=ticker.run).start()
        writer.commit()
        writer.close()
        #ticker.tick = False
        ##print ('done')

    def indexDocs(self, root, writer):

        t1 = FieldType()
        t1.setStored(True)
        t1.setTokenized(False)
        t1.setStoreTermVectors(True)
        t1.setStoreTermVectorOffsets(True)
        t1.setStoreTermVectorPositions(True)
        t1.setIndexOptions(IndexOptions.DOCS_AND_FREQS_AND_POSITIONS_AND_OFFSETS)
        

        t2 = FieldType()
        t2.setStored(True)
        t2.setTokenized(True)
        t2.setStoreTermVectors(True)
        t2.setStoreTermVectorOffsets(True)
        t2.setStoreTermVectorPositions(True)
        t2.setIndexOptions(IndexOptions.DOCS_AND_FREQS_AND_POSITIONS_AND_OFFSETS)
                
        i = 1
        for index, row in final_df.iterrows():
            print ("adding ", i , "th document:", row['cord_uid'])
            try :
                doc = Document()
                doc.add(Field("cord_uid", row['cord_uid'], t1))
                doc.add(Field("title", row['title'], t2))
                doc.add(Field("doi",row['doi'], t1))
                doc.add(Field("pmcid", row['pmcid'], t1))
                doc.add(Field("publish_time", row['publish_time'], t1))
                doc.add(Field("journal", row['journal'], t1))
                doc.add(Field("url", row['url'], t1))
                
                if len(row['abstract']) > 0:
                    doc.add(Field("full_text", row['abstract'], t2))
                else :
                    print ("warning: no fulltext available in %s", row['title'])
                    
#                 if len(row['abstract_y']) > 0:
#                     doc.add(Field("abstract", row['abstract_y'], t2))
#                 else :
#                     print ("warning: no abstract available in %s", row['title'])
                writer.addDocument(doc)
            except (RuntimeError, TypeError, NameError):
                pass
            i=i+1
            
            
lucene.initVM()
start = datetime.now()
try:
    IndexFiles(LUCENE_BASE_DIR, os.path.join(LUCENE_BASE_DIR, LUCENE_INDEX_DIR),StandardAnalyzer())
    end = datetime.now()
    print (end - start)
except (RuntimeError, TypeError, NameError):
    print ("Failed: ")
    raise        

In [ ]:
# static_risk_factors = pd.DataFrame(["sex","gender","male","cancer","cerebral","hypertension", "smok", "respiratory", 
#             "pneumonia", "vascular", "influenza", "tobacco", "obes", "virus",
#             "cardio", "bronchitis"], columns=["name"])

In [ ]:
static_risk_factors = pd.DataFrame(["diagnostics","findings","future","discover","advances","development", "X-ray", "symptoms", 
            "scan", "antibodies", "RT-PCR", "viral test", "viral", "evidence",
            "studies show", "sample","COVID","test","diagnostic tools","corona","diagnostics"], columns=["name"])

In [ ]:
TOP_K = 20
df_relevant_papers = []

def run(searcher, analyzer, search_topic):
    if search_topic == '':
        return

#     print ("Searching for:", search_topic)
    query = QueryParser("full_text", analyzer).parse(search_topic)
    #print('---------query-------',query)

    scoreDocs = searcher.search(query, TOP_K).scoreDocs
    print ("%s total matching documents." , len(scoreDocs))
    
    for scoreDoc in scoreDocs:
        doc = searcher.doc(scoreDoc.doc)
        result = {
            'date': doc.get("publish_time"),
            'title': doc.get("title"),
            'url': doc.get("url"),
            'journal': doc.get("journal"),
            'cord_uid': doc.get('cord_uid'),
            'abstract': doc.get('abstract'),
            #'risk_factor': risk_factor,
            'pmcid': doc.get("pmcid")
        }

        df_relevant_papers.append(result)                

directory = SimpleFSDirectory(Paths.get(os.path.join(LUCENE_BASE_DIR, LUCENE_INDEX_DIR)))
searcher = IndexSearcher(DirectoryReader.open(directory))
analyzer = StandardAnalyzer()

PROXIMITY = 5
for id, item in static_risk_factors.iterrows():
    diagnostics_factor = item['name']
    #search_topic = ("\"{}\" AND (\"coronavirus disease 19\" OR \"sars cov 2\" OR \"2019 ncov\" OR \"2019ncov\" OR \"coronavirus 2019\", \"wuhan pneumonia\" OR \"wuhan virus\"OR \"wuhan coronavirus\",OR \"covid19\" OR \"covid-19\")".format(diagnostics_factor))
    #search_topic = ("(\"clinical\" OR \"epidem\" OR \"virological\" OR \"host susceptibility to serve\" OR \"coronavirus disease 19\") AND \"characteristic\" AND (\"coronavirus disease 19\" OR \"sars cov 2\" OR \"2019 ncov\" OR \"2019ncov\" OR \"coronavirus 2019\", \"wuhan pneumonia\" OR \"wuhan virus\"OR \"wuhan coronavirus\",OR \"covid19\" OR \"covid-19\")")
    search_topic = ("\"{}\" AND (\"covid\" OR \"sars cov 2\"  OR \"coronavirus 2019\",\"wuhan pneumonia\" OR \"wuhan virus\"OR \"wuhan coronavirus\",OR \"covid19\" OR \"covid-19\")".format(diagnostics_factor))
    print(diagnostics_factor)
    print('search_topic: ', search_topic)    
    run(searcher, analyzer, search_topic)
del searcher

df_relevant_papers = pd.DataFrame(df_relevant_papers)
df_relevant_papers = df_relevant_papers.drop_duplicates()
len(df_relevant_papers)

In [ ]:
len(df_relevant_papers)

In [ ]:
df_relevant_papers.head()

In [ ]:
df_relevant_papers['url']

In [ ]:
url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id={0}'.format('PMC2769550')

In [ ]:
url

In [ ]:
source = urllib.request.urlopen(url).read()

In [ ]:

soup = bs.BeautifulSoup(source,'lxml')
df = None
cols = None
dfs = []
title = ''
abstract = ''
journal = ''
title = soup.findAll('article-title')[0].find(text=True)# title
    

In [ ]:
#title = soup.findAll('article-title')[0].find(text=True)# title

In [ ]:
get_all_tables('PMC2769550')

In [ ]:
# This method will display all tables in a paper and will return them as a list of dfs
def get_all_tables(pmcid, show_tables = True):
#     print("", end = '.')
    sleep(0.5) # This line is needed as the NCBI server will block if there are too many frequent requests
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id={0}'.format(pmcid)
    #beautiful soup
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source,'lxml')
    df = None
    cols = None
    dfs = []
    title = ''
    abstract = ''
    journal = ''
    title = soup.findAll('article-title')[0].find(text=True)# title
    try:
        abstract = soup.find("abstract").text
    except:
        abstract = ''
    #getting the table
    tables = soup.findAll('table')  # Note: 
    i = 1
    for table in tables:
        if(show_tables):
            print("\nTable {0}  for paper id {1}: ".format(i, pmcid))
        table_rows = table.find_all('tr')
        #generate df
        df = pd.DataFrame()
        for tr in table_rows:
            td = tr.find_all(['td'])
            row = [i.text for i in td]
            
            if(len(row) > 0):
                s = pd.Series(row)
                df = df.append(s, ignore_index=True)
            else: # it is a header
                th = tr.find_all(['th'])
                cols = [i.text for i in th]
        
        if(cols): #assign header
            if(len(df.columns) == len(cols)+1):
                #add one column for the title
                cols.insert(0,'')
            if(len(df.columns) == len(cols)):
                df.columns = cols
        
        dfs.append( df )
        if(show_tables):
            print(title)
            print(abstract)
            display(HTML(df.to_html()))
        i = i + 1

    return dfs

In [ ]:
%%capture
# This section loads the indexs and takes about 4 minutes to run
class Ticker(object):

    def __init__(self):
        self.tick = True

    def run(self):
        while self.tick:
            sys.stdout.write('.')
            sys.stdout.flush()
            time.sleep(1.0)

class IndexFiles(object):
    """Usage: python IndexFiles <doc_directory>"""

    def __init__(self, root, storeDir, analyzer):
        ##print("before store")
        if not os.path.exists(storeDir):
            os.mkdir(storeDir)
        ##print("after store")

        store = SimpleFSDirectory(Paths.get(storeDir))
        ##print(storeDir)
        analyzer = LimitTokenCountAnalyzer(analyzer, 1048576)
        ##print("after analyzer ")

        config = IndexWriterConfig(analyzer)
        ##print("after config")

        config.setOpenMode(IndexWriterConfig.OpenMode.CREATE)
        ##print("before writer")
        writer = IndexWriter(store, config)
        ##print("after writer")
        self.indexDocs(root, writer)
        ticker = Ticker()
        ##print ('commit index')
        threading.Thread(target=ticker.run).start()
        writer.commit()
        writer.close()
        ticker.tick = False
        ##print ('done')

    def indexDocs(self, root, writer):

        t1 = FieldType()
        t1.setStored(True)
        t1.setTokenized(False)
        t1.setStoreTermVectors(True)
        t1.setStoreTermVectorOffsets(True)
        t1.setStoreTermVectorPositions(True)
        t1.setIndexOptions(IndexOptions.DOCS_AND_FREQS_AND_POSITIONS_AND_OFFSETS)
        

        t2 = FieldType()
        t2.setStored(True)
        t2.setTokenized(True)
        t2.setStoreTermVectors(True)
        t2.setStoreTermVectorOffsets(True)
        t2.setStoreTermVectorPositions(True)
        t2.setIndexOptions(IndexOptions.DOCS_AND_FREQS_AND_POSITIONS_AND_OFFSETS)
                
        i = 1
        for index, row in final_df.iterrows():
            print ("adding ", i , "th document:", row['paper_id'])
            try :
                doc = Document()
                doc.add(Field("paper_id", row['paper_id'], t1))
                doc.add(Field("title", row['title'], t2))
                doc.add(Field("doi",row['doi'], t1))
                doc.add(Field("pmcid", row['pmcid'], t1))
                doc.add(Field("publish_time", row['publish_time'], t1))
                doc.add(Field("journal", row['journal'], t1))
                doc.add(Field("url", row['url'], t1))
                
                if len(row['text']) > 0:
                    doc.add(Field("full_text", row['text'], t2))
                else :
                    print ("warning: no fulltext available in %s", row['title'])
                    
                if len(row['abstract_y']) > 0:
                    doc.add(Field("abstract", row['abstract_y'], t2))
                else :
                    print ("warning: no abstract available in %s", row['title'])
                writer.addDocument(doc)
            except (RuntimeError, TypeError, NameError):
                pass
            i=i+1
            

lucene.initVM()
start = datetime.now()
try:
    IndexFiles(LUCENE_BASE_DIR, os.path.join(LUCENE_BASE_DIR, LUCENE_INDEX_DIR),StandardAnalyzer())
    end = datetime.now()
    print (end - start)
except (RuntimeError, TypeError, NameError):
    print ("Failed: ")
    raise 

### Search String
This sub-section finds the most relevant papers related to risk factors. The dataframe *static_risk_factors* is used, but can be updated as needed:

"sex","gender","male","cancer","cerebral","hypertension", "smok", "respiratory", "pneumonia", "vascular", "influenza", "tobacco", "obes", "virus", "cardio", "bronchitis"


In [ ]:
static_risk_factors = pd.DataFrame(["sex","gender","male","cancer","cerebral","hypertension", "smok", "respiratory", 
            "pneumonia", "vascular", "influenza", "tobacco", "obes", "virus",
            "cardio", "bronchitis"], columns=["name"])

In [ ]:
TOP_K = 200
df_relevant_papers = []

def run(searcher, analyzer, search_topic):
    if search_topic == '':
        return

#     print ("Searching for:", search_topic)
    query = QueryParser("full_text", analyzer).parse(search_topic)

    scoreDocs = searcher.search(query, TOP_K).scoreDocs
    print ("%s total matching documents." , len(scoreDocs))
    
    for scoreDoc in scoreDocs:
        doc = searcher.doc(scoreDoc.doc)
        result = {
            'date': doc.get("publish_time"),
            'study': doc.get("title"),
            'study_link': doc.get("url"),
            'journal': doc.get("journal"),
            'paper_id': doc.get('paper_id'),
            'paper_full_text': doc.get('full_text'),
            #'risk_factor': risk_factor,
            'pmcid': doc.get("pmcid")
        }

        df_relevant_papers.append(result)                

directory = SimpleFSDirectory(Paths.get(os.path.join(LUCENE_BASE_DIR, LUCENE_INDEX_DIR)))
searcher = IndexSearcher(DirectoryReader.open(directory))
analyzer = StandardAnalyzer()

PROXIMITY = 20
for id, item in static_risk_factors.iterrows():
    risk_factor = item['name']
    #search_topic = ("\"{}\" AND (\"coronavirus disease 19\" OR \"sars cov 2\" OR \"2019 ncov\" OR \"2019ncov\" OR \"coronavirus 2019\", \"wuhan pneumonia\" OR \"wuhan virus\"OR \"wuhan coronavirus\",OR \"covid19\" OR \"covid-19\")".format(risk_factor))
    search_topic = ("(\"clinical\" OR \"epidem\" OR \"virological\" OR \"host susceptibility to serve\") AND \"characteristic\" AND (\"coronavirus disease 19\" OR \"sars cov 2\" OR \"2019 ncov\" OR \"2019ncov\" OR \"coronavirus 2019\", \"wuhan pneumonia\" OR \"wuhan virus\"OR \"wuhan coronavirus\",OR \"covid19\" OR \"covid-19\")")
    print(risk_factor)
    print('search_topic: ', search_topic)    
    run(searcher, analyzer, search_topic)
del searcher

df_relevant_papers = pd.DataFrame(df_relevant_papers)
df_relevant_papers = df_relevant_papers.drop_duplicates()
len(df_relevant_papers)

In [ ]:
# Examine results 
df_relevant_papers.head()

[](http://)Filtering articles that have a PMCID, as these publications store tabular data on PubMed.


In [ ]:
# Only use papers that have a PMCID
df_relevant_papers = df_relevant_papers[~df_relevant_papers['pmcid'].str.contains('NaN')]

In [ ]:
# This list will be used to pull publication data from PubMed servers
relevant_papers = df_relevant_papers.pmcid.values

## Load Tables <a class="anchor" id="xml-header"></a>
![](http://)
This section will retrieve tables from the publications. It will then parse the XML table data to retrieve relevant information.

In [ ]:
# Load Kaggle Metadata
metadata = pd.read_csv('../input/CORD-19-research-challenge/metadata.csv')

### Extract Information from Tables <a class="anchor" id="manual-header"></a>
![](http://)
We are relying on the search above to identify the most relevant tables.
This section will get information from the extracted tables related to a given disease. 

** However, this works best for a fixed set of consistent table styles.** The system can be updated in the future to accommodate more diverse tables.

In [ ]:
#Input: the dataframe you are looking for and the set of synonyms to search in the header
#Output: the indexes where the non severe and severe cases are described, in this order.
def get_column_index_severities(df, severe_synonyms_set):
    
    synonyms_non = set(["non", "non ","non-","no ","dead","death","died","3","mort"])

    row = list(df.columns)
    
    severe_index = None
    non_severe_index = None
    
    severe_value = ""
    non_severe_value = ""

    for i, r in enumerate(row):
        r = str(r).lower().replace('‐', '-')
        #print("r is",r)
        for sv in severe_synonyms_set:
            if(sv in r):   
                #print("Checking",sv)
                #check if its not severe
                severe_value = sv
                for n in synonyms_non:
                    #print("Checking",sv," and",n)
                    if(n+sv in r) or (n in r):
                        non_severe_index = i
                        non_severe_value = n
                        #print("n"+n)
                        break
                
                if(not non_severe_index):
                    severe_index = i

                if(severe_index and non_severe_index):
                    #print("Found index for " + severe_value + " and " + non_severe_value)
                    return [non_severe_index, severe_index]                
                
                break #not necessary to look for more synonyms on severe set
    return [non_severe_index, severe_index]

> ### NOTE:
With a pmcid, it is possible to get download tables from studies in XML format.

In [ ]:
def get_only_number(val):
    if isinstance(val, str):
        i = val.find("(")
        if(i > 0):
            val = val[0:i]
        i = val.find("/")
        if(i > 0):
            val = val[0:i]
    try:
        f = float(val)
        return f
    except:
        return 0

def get_total_number(header):
    
    val = header[header.find("=")+1:header.find(")")].strip() #pattern (n = 12)
    if(val.isdigit()):
        return int(val)
    return 0
    

def get_numbers_by_disease(df, disease, header_synonyms):
    disease = disease.lower()
    non_severe_index, severe_index = get_column_index_severities(df, header_synonyms)
    
    if(not non_severe_index or not severe_index):
        # print("Info: Severity not found on header")
        return
        
    # get row
    row = df[df.iloc[:,0].str.lower().str.contains(disease)].values
    if(len(row) == 0): #did not find the disease
         # print("Info: Disease not found")
        return 
    row = row[0]
    
    header = list(df.columns)

    if(non_severe_index):
        non_severe = get_only_number(row[non_severe_index])
        non_severe_n = get_total_number(header[non_severe_index])


    if(severe_index):
        severe = get_only_number(row[severe_index])
        severe_n = get_total_number(header[severe_index])


    return [non_severe, non_severe_n, severe, severe_n]


def get_severity_numbers(df, disease):
    synonyms_severe = set(["severe", "critical", "icu", "picu","poor","mild","ecmo", "mv","gi","corticosteroids","2"])
    return get_numbers_by_disease(df, disease, synonyms_severe)

def get_fatality_numbers(df, disease):
    synonyms_fatality = set(["surviv"])
    return get_numbers_by_disease(df, disease, synonyms_fatality)

In [ ]:
# This method will display all tables in a paper and will return them as a list of dfs
def get_all_tables(pmcid, show_tables = True):
#     print("", end = '.')
    sleep(0.5) # This line is needed as the NCBI server will block if there are too many frequent requests
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id={0}'.format(pmcid)
    #beautiful soup
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source,'lxml')
    df = None
    cols = None
    dfs = []
    title = ''
    abstract = ''
    journal = ''
    title = soup.findAll('article-title')[0].find(text=True)# title
    try:
        abstract = soup.find("abstract").text
    except:
        abstract = ''
    #getting the table
    tables = soup.findAll('table')  # Note: 
    i = 1
    for table in tables:
        if(show_tables):
            print("\nTable {0}  for paper id {1}: ".format(i, pmcid))
        table_rows = table.find_all('tr')
        #generate df
        df = pd.DataFrame()
        for tr in table_rows:
            td = tr.find_all(['td'])
            row = [i.text for i in td]
            
            if(len(row) > 0):
                s = pd.Series(row)
                df = df.append(s, ignore_index=True)
            else: # it is a header
                th = tr.find_all(['th'])
                cols = [i.text for i in th]
        
        if(cols): #assign header
            if(len(df.columns) == len(cols)+1):
                #add one column for the title
                cols.insert(0,'')
            if(len(df.columns) == len(cols)):
                df.columns = cols
        
        dfs.append( df )
        if(show_tables):
            print(title)
            print(abstract)
            display(HTML(df.to_html()))
        i = i + 1

    return dfs

## Compute Odds Ratio <a class="anchor" id="odds-header"></a>
![](http://)
As we are analysing count data, such as the number of severe outcomes versus the number of non-severe outcomes, we will compute the summary statistics using an odds ratio. 
The term “odds” refers to the probability of an event occurring versus the probability of the event not occurring.

These can be illustrated with the following example, based on [Ranganathan](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4640017):

| | Fatal | Survive | Total |
| --- | --- | --- | --- |
| A | 18 | 46 | 64 |
| B | 29 | 36 | 65 |
| Totals | 47 | 82 | 129 |

Odds is defined as:

$$Odds = \frac{Fatal}{Survive}$$

The odds of death in group A is 18/46 (0.39) and in group B it is 29/36 (0.81).

The odds *ratio* (OR) is the ratio of the odds of an event in group A to the odds of an event in group B. 

Our Covid-19 data is mainly from retrospective cohort studies, which are studies that examine existing data to identify risk factors for particular conditions, therefore we will utilise  odds  ratio.

In [ ]:
def get_statistical_info(severity_numbers):
#severity_number pattern: non-severity, non-severity_n, severity, severity_n
    OR, lower_CI, higher_CI, CI, p_value = 0.0, 0.0, 0.0, 0.0, 0.0
    ##Cases with positive (bad) outcome
    a = severity_numbers[2]     # Number in exposed group
    b = severity_numbers[3] - a # Number in control group

    ##Cases with negative (good) outcome
    c = severity_numbers[0]     # Number in exposed group
    d = severity_numbers[1] - c # Number in control group
    
    if(a == 0 or b <= 0 or c == 0 or d <=0): # cannot calculate division by 0 or sqrt < 0
        return OR, lower_CI, higher_CI, CI, p_value
    
    
    OR = (a*d)/(b*c)
    std_error  = np.sqrt(1/a + 1/b + 1/c +1/d) 
    
    lower_CI = np.exp(np.log(OR)-1.96*std_error)
    higher_CI = np.exp(np.log(OR)+1.96*std_error)
    
    CI = "95% CI: {:.2f}-{:.2f}".format(lower_CI, higher_CI)
    
    #P-value: φ(z) = (1 / √2π) × e -z2/2
    z = np.log(OR) / std_error
    
    p_value = (1/(np.sqrt(2*np.pi))) * np.exp(-(z**2)/2)
    
    #return "OR: {:.2f} ({:}) p={:.2f}".format(OR, CI, p_value)
    return OR, lower_CI, higher_CI, CI, p_value
    

## Display Results <a class="anchor" id="display-header"></a>
![](http://)
Display the results and save findings in CSV files.

In [ ]:
def generate_csv_file(tables_by_pmcid, disease):
# Generates a CSV file for risk factor, comorbidity (disease) in line with target format
    result_df = pd.DataFrame(columns=['Risk','Date', 'Title','url','Journal','Severe','Severe Lower Bound','Severe Upper Bound','Severe p-value','Severe Significant'])

    for t in tables_by_pmcid:
        pmcid = t['pmcid']
        paper_data_dict = {}
        #get information from the metadata
        paper_info = metadata[metadata.pmcid == pmcid][['title', 'publish_time', 'source_x', 'url', 'pmcid','journal']]
        if(len(paper_info) == 0):
            #if does not find the paper, don't continue the process
            break

        paper_info = paper_info.iloc[0]
        paper_data_dict['Risk']   = disease
        paper_data_dict['Date']   = paper_info.publish_time
        paper_data_dict['Title']  = paper_info.title
        paper_data_dict['url']    = paper_info.url
        paper_data_dict['Journal']    = paper_info.journal
        
        fatality_numbers = None
        severe_numbers = None
        for df in t['df_list']:
            df.columns = df.iloc[0]
            f = get_fatality_numbers(df, disease)
            s = get_severity_numbers(df, disease)

            if(f): fatality_numbers = f
            if(s): severe_numbers = s
         

        if(severe_numbers): 
            #try:
            severe_OR, severe_lower_CI, severe_higher_CI, severe_CI, severe_p_value = get_statistical_info(severe_numbers)
            paper_data_dict['Severe'] = severe_OR
            paper_data_dict['Severe Lower Bound'] = severe_lower_CI
            paper_data_dict['Severe Upper Bound'] = severe_higher_CI
            paper_data_dict['Severe p-value'] = severe_p_value
            if (float(severe_p_value)<0.05):
                paper_data_dict['Severe Significant'] = "Significant"
            else:
                paper_data_dict['Severe Significant'] = "Not Significant"
            #paper_data_dict['Counts'] = str(severe_numbers[1]) + "," + str(severe_numbers[3])
            #paper_data_dict['Sample'] = severe_numbers[1] + severe_numbers[3] 
            #except:
                # No data  
                #print("Insuffient data")

        if('Severe' in paper_data_dict): #or ('Fatality' in paper_data_dict):
            result_df = result_df.append(paper_data_dict, ignore_index=True)
            break
    
    if(len(result_df) > 0):
        result_df.to_csv(disease + ".csv")
        display(HTML(result_df.to_html()))
    return result_df

In [ ]:
def generate_summary_csv_file(tables_by_pmcid, disease):
# Generates a summary CSV file for risk factor, comorbidity (disease)
    result_df = pd.DataFrame(columns=['Risk','Date', 'Study', 'pmcid','Source','url','Severe','Counts','Fatality', 'Sample'])
    for t in tables_by_pmcid:
        pmcid = t['pmcid']
        paper_data_dict = {}
        #get information from the metadata
        paper_info = metadata[metadata.pmcid == pmcid][['title', 'publish_time', 'source_x', 'url', 'pmcid']]
        if(len(paper_info) == 0):
            #if does not find the paper, don't continue the process
            break

        paper_info = paper_info.iloc[0]
        paper_data_dict['Risk']   = disease
        paper_data_dict['Date']   = paper_info.publish_time
        paper_data_dict['Study']  = paper_info.title
        paper_data_dict['pmcid']    = paper_info.pmcid
        paper_data_dict['Source'] = paper_info.source_x
        paper_data_dict['url']    = paper_info.url

        fatality_numbers = None
        severe_numbers = None
        for df in t['df_list']:
            df.columns = df.iloc[0]
            f = get_fatality_numbers(df, disease)
            s = get_severity_numbers(df, disease)
            if(f): fatality_numbers = f
            if(s): severe_numbers = s
                
        if(fatality_numbers): 
            #f = get_statistical_info(fatality_numbers)
            fatal_OR, fatal_lower_CI, fatal_higher_CI, fatal_CI, fatal_p_value = get_statistical_info(fatality_numbers)
            paper_data_dict['Fatality'] = "OR: {:.2f} ({:}) p={:.2f}".format(fatal_OR, fatal_CI, fatal_p_value)
            paper_data_dict['Sample'] = fatality_numbers[1] + fatality_numbers[3] 

        if(severe_numbers): 
            #s = get_statistical_info(fatality_numbers)
            severe_OR, severe_lower_CI, severe_higher_CI, severe_CI, severe_p_value = get_statistical_info(severe_numbers)
            paper_data_dict['Severe'] = "OR: {:.2f} ({:}) p={:.2f}".format(severe_OR, severe_CI, severe_p_value)
            paper_data_dict['Counts'] = str(severe_numbers[1]) + "," + str(severe_numbers[3])
            paper_data_dict['Sample'] = severe_numbers[1] + severe_numbers[3] 

                
        if('Severe' in paper_data_dict) or ('Fatality' in paper_data_dict):
            result_df = result_df.append(paper_data_dict, ignore_index=True)
            break
    
    if(len(result_df) > 0):
        result_df.to_csv(disease + "-summary.csv")
        display(HTML(result_df.to_html()))
    return result_df

In [ ]:
#getting all tables from the relevant papers
tables_by_pmcid = []
for pmcid in relevant_papers:
    df_list = get_all_tables(pmcid, False)
    if(len(df_list) > 0): tables_by_pmcid.append({'pmcid': pmcid, 'df_list': df_list})

In [ ]:
# Print some stats on tables 
m = 0
n = 0
for paper in tables_by_pmcid:
    if paper is not None:
        m += len(paper['df_list'])
        n += 1
print('Average number of tables per paper: {0}'.format(1.0*m/n))
print('Number of papers with tables: {0}'.format(n))

In [ ]:
# Generate target tables
diseases = static_risk_factors.name.values.tolist()
for d in diseases:
    #print("\nTable for disease",d)
    generate_csv_file(tables_by_pmcid, d)

In [ ]:
# Generate summary tables
diseases = static_risk_factors.name.values.tolist()
for d in diseases:
    #print("\nTable for disease",d)
    generate_summary_csv_file(tables_by_pmcid, d)

## Conclusion <a class="anchor" id="conclusion-header"></a>

We set out to provide a reproducible pipeline to search and extract quantitative, formal, data from previous research studies to derive odds ratios and confidence intervals for risk factors and comorbidities for Covid-19. 

### Pros and Cons of this Approach

#### Pros
Our approach provides the following features:
* We combine high performance indexing and search features with online web services to extract quantitative data related to Covid-19. 
* We automate all of the steps needed to extract this crucial information in one reusable pipeline.
* The end result is a set of odds ratios and confidence interval that provide statistical insight into Covid-19 risk factors.
* The aggregation of data from multiple risk factor studies, can lead to a higher statistical power and more robust estimates than is possible from any individual study.
* The analysis is highly reproducible, allowing other researchers to replicate and reuse the pipeline.
* The system can be re-run as more data becomes available.
    
#### Cons 
The following are the shortfalls of our approach:
* We are pulling tables automatically where they have been provided in XML. We can expand the technique to HTML and PDF tables in the future.
* The technique could suffer from publication bias, which is a type of bias that occurs when publishers favour results that show significant findings, potentially allowing the results to be generalized to a larger population.

### Future Work
* Expand the technique to HTML and PDF tables.
* Add meta-analysis.
* Inconsistency of results across studies can be quantified and analyzed.
* Compute risk ratios
* Process more table types.